In [1]:
#Import packages
import numpy as np
import pandas as pd
import json
import datetime
from collections import defaultdict

In [2]:
#Load data
with open("data/receipts.json") as receipts_file:
    receipts_raw = [json.loads(line) for line in receipts_file]
with open("data/users.json") as users_file:
    users_raw = [json.loads(line) for line in users_file]
with open("data/brands.json") as brands_file:
    brands_raw = [json.loads(line) for line in brands_file]

## Initial exploration of Receipts Data schema

First, I'd like to take a look at a handful of example data points from each schema.

In [3]:
print(json.dumps(receipts_raw[1:4],indent=2))

[
  {
    "_id": {
      "$oid": "5ff1e1bb0a720f052300056b"
    },
    "bonusPointsEarned": 150,
    "bonusPointsEarnedReason": "Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",
    "createDate": {
      "$date": 1609687483000
    },
    "dateScanned": {
      "$date": 1609687483000
    },
    "finishedDate": {
      "$date": 1609687483000
    },
    "modifyDate": {
      "$date": 1609687488000
    },
    "pointsAwardedDate": {
      "$date": 1609687483000
    },
    "pointsEarned": "150.0",
    "purchaseDate": {
      "$date": 1609601083000
    },
    "purchasedItemCount": 2,
    "rewardsReceiptItemList": [
      {
        "barcode": "4011",
        "description": "ITEM NOT FOUND",
        "finalPrice": "1",
        "itemPrice": "1",
        "partnerItemId": "1",
        "quantityPurchased": 1
      },
      {
        "barcode": "028400642255",
        "description": "DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ",
        "finalPric

#### Observations about these example data points:

* Value of $oid within the "_id" node identifies the receipt
* Dates (createDate, dateScanned, finishedDate, and modifyDate appear to be in milliseconds since 1-1-1970, but this should be confirmed with the stakeholder:

In [4]:
print(datetime.datetime.fromtimestamp(receipts_raw[0]["dateScanned"]["$date"]/1000.0))

2021-01-03 09:25:31


* pointsEarned is in string format with one decimal place, but bonusPointsEarned is in int format.  This should be considered when choosing data types for database tables
* purchasedItemCount does *not* necessarily match the number of items in rewardsReceiptItemList
* totalSpent does not match the sum of userFlaggedPrice for the receipt item that is missing an itemPrice
* rewardsReceiptItemList is really what we should be looking at here
* userID can be used as a foreign key to the Users data schema
* Items might be not found, with barcode 4011
* brandCode can be used as a foreign key to the Brands data schema
* brandCode is not populated for every receipt line  


Questions for data analyst:
* Which keys have missing values across the entire dataset?
* What is the total count of and amount spent on receipt lines with missing or 4011 bar codes?
* Does the totalSpent 

Question for stakeholder:
* What should be done with receipt lines with a barcode of 4011 and/or a missing brand code?
* Is item price, final price, or user flagged price the source of truth for the amount spent on an individual item?


### Initial exploration of Users Data schema

In [5]:
print(json.dumps(users_raw[0:4],indent=2))

[
  {
    "_id": {
      "$oid": "5ff1e194b6a9d73a3a9f1052"
    },
    "active": true,
    "createdDate": {
      "$date": 1609687444800
    },
    "lastLogin": {
      "$date": 1609687537858
    },
    "role": "consumer",
    "signUpSource": "Email",
    "state": "WI"
  },
  {
    "_id": {
      "$oid": "5ff1e194b6a9d73a3a9f1052"
    },
    "active": true,
    "createdDate": {
      "$date": 1609687444800
    },
    "lastLogin": {
      "$date": 1609687537858
    },
    "role": "consumer",
    "signUpSource": "Email",
    "state": "WI"
  },
  {
    "_id": {
      "$oid": "5ff1e194b6a9d73a3a9f1052"
    },
    "active": true,
    "createdDate": {
      "$date": 1609687444800
    },
    "lastLogin": {
      "$date": 1609687537858
    },
    "role": "consumer",
    "signUpSource": "Email",
    "state": "WI"
  },
  {
    "_id": {
      "$oid": "5ff1e1eacfcf6c399c274ae6"
    },
    "active": true,
    "createdDate": {
      "$date": 1609687530554
    },
    "lastLogin": {
      "$date": 160

#### Observations about these example data points:

* They are duplicates - we should check other files for the same issue
* Dates are also in the milliseconds since 1-1-1970 (probably) format
* $oid does exist in receipts.json, so I plan to use it as the link between the two schema 
* What's the point of the "role" value if it is always "consumer"?
* signUpSource is not mentioned in the Users data schema description in the exercise
* Nothing else major of note

Questions for stakeholder:
* Is it possible, at some point in the future, data will be added to the Users dataset with a role other than "consumer"?
* Is the signUpSource of individual users relevant for your business needs?

### Initial exploration of Brand Data schema

In [6]:
print(json.dumps(brands_raw[1:4],indent=2))

[
  {
    "_id": {
      "$oid": "601c5460be37ce2ead43755f"
    },
    "barcode": "511111519928",
    "brandCode": "STARBUCKS",
    "category": "Beverages",
    "categoryCode": "BEVERAGES",
    "cpg": {
      "$id": {
        "$oid": "5332f5fbe4b03c9a25efd0ba"
      },
      "$ref": "Cogs"
    },
    "name": "Starbucks",
    "topBrand": false
  },
  {
    "_id": {
      "$oid": "601ac142be37ce2ead43755d"
    },
    "barcode": "511111819905",
    "brandCode": "TEST BRANDCODE @1612366146176",
    "category": "Baking",
    "categoryCode": "BAKING",
    "cpg": {
      "$id": {
        "$oid": "601ac142be37ce2ead437559"
      },
      "$ref": "Cogs"
    },
    "name": "test brand @1612366146176",
    "topBrand": false
  },
  {
    "_id": {
      "$oid": "601ac142be37ce2ead43755a"
    },
    "barcode": "511111519874",
    "brandCode": "TEST BRANDCODE @1612366146051",
    "category": "Baking",
    "categoryCode": "BAKING",
    "cpg": {
      "$id": {
        "$oid": "601ac142be37ce2ead437559"

#### Observations/questions about these example data points:

* Three data points are from a test brand.  Test data points should be removed.
* The oid within cpg appears to correspond to rewardsProductPartnerID in the rewardsReceiptItemLIst, but it is not unique within brands.json
* What are the barcodes on each brand?  They do not appear to match the barcodes in the receipt item lines

Questions for stakeholder:

* How can we reliably identify and remove test brands from the brands dataset?


## Evaluation of data quality

First, I'd like to evaluate the completeness of the data in each schema.  I'll approach this by finding the proportion of each field that is populated in each schema, along with total counts and data types:

In [7]:
def dataCompletenessReport(list_of_dicts):
    """
    Accepts a list of dictionaries and returns a pandas DataFrame summarizing
    the fields in the dictionaries, data types, data completeness,
    and a few other characteristics
    Assumes:  Dictionaries in the input list have keys in common

    """
    key_count_dict = defaultdict(int)
    n_lines = len(list_of_dicts)
    datatype_dict = defaultdict(set)
    for curr in list_of_dicts: #loop over raw data    
        for key,value in curr.items():
            key_count_dict[key]+=1 #count number of rows with this key populated
            curr_type = type(value)
            datatype_dict[key].add(curr_type)
    dataCompDf = pd.DataFrame([datatype_dict,key_count_dict]).T
    dataCompDf.columns = ["Data Types","Count Populated"]
    dataCompDf["Proportion Populated"] = dataCompDf["Count Populated"]/n_lines

    return dataCompDf

In [8]:
dataCompletenessReport(receipts_raw)

,Data Types,Count Populated,Proportion Populated
_id,{<class 'dict'>},1119,1.0
bonusPointsEarned,{<class 'int'>},544,0.486148
bonusPointsEarnedReason,{<class 'str'>},544,0.486148
createDate,{<class 'dict'>},1119,1.0
dateScanned,{<class 'dict'>},1119,1.0
finishedDate,{<class 'dict'>},568,0.507596
modifyDate,{<class 'dict'>},1119,1.0
pointsAwardedDate,{<class 'dict'>},537,0.479893
pointsEarned,{<class 'str'>},609,0.544236
purchaseDate,{<class 'dict'>},671,0.599643


#### Observations about Receipts data completeness:

* rewardsReceiptItemList has type list.  We should investigate data completeness of it as well.
* The date points were awarded is not populated for every line where points were earned.
* Total spent, item list, and item count are not populated for every receipt.

In [9]:
#Build a list of raw receipt item lines to check for data completeness and quality
rewardsReceiptItemLines_raw = [curr_receipt.get('rewardsReceiptItemList') for curr_receipt in receipts_raw if curr_receipt.get('rewardsReceiptItemList') is not None]
rewardsReceiptItemLines_flat = [curr_receipt for curr_list in rewardsReceiptItemLines_raw for curr_receipt in curr_list]
dataCompletenessReport(rewardsReceiptItemLines_flat)

,Data Types,Count Populated,Proportion Populated
barcode,{<class 'str'>},3090,0.445181
description,{<class 'str'>},6560,0.945109
finalPrice,{<class 'str'>},6767,0.974932
itemPrice,{<class 'str'>},6767,0.974932
needsFetchReview,{<class 'bool'>},813,0.11713
partnerItemId,{<class 'str'>},6941,1.0
preventTargetGapPoints,{<class 'bool'>},358,0.051578
quantityPurchased,{<class 'int'>},6767,0.974932
userFlaggedBarcode,{<class 'str'>},337,0.048552
userFlaggedNewItem,{<class 'bool'>},323,0.046535


#### Observations about rewardsReceiptItemList data completeness:
* brandCode is only populated for 37.45% of receipt lines
* barcode is not populated for every receipt line
* Approximately 2% of items are missing item prices

Data quality question to investigate:
* What proportion of receipt lines and total spend is in receipt lines with a 4011 barcode or a missing brand code?

In [10]:
dataCompletenessReport(users_raw)

,Data Types,Count Populated,Proportion Populated
_id,{<class 'dict'>},495,1.0
active,{<class 'bool'>},495,1.0
createdDate,{<class 'dict'>},495,1.0
lastLogin,{<class 'dict'>},433,0.874747
role,{<class 'str'>},495,1.0
signUpSource,{<class 'str'>},447,0.90303
state,{<class 'str'>},439,0.886869


#### Observations about Users data completeness:
* signUpSource is not mentioned in the schema
* There isn't much of note or concern relating to data completeness for Users.

In [11]:
dataCompletenessReport(brands_raw)

,Data Types,Count Populated,Proportion Populated
_id,{<class 'dict'>},1167,1.0
barcode,{<class 'str'>},1167,1.0
category,{<class 'str'>},1012,0.867181
categoryCode,{<class 'str'>},517,0.443016
cpg,{<class 'dict'>},1167,1.0
name,{<class 'str'>},1167,1.0
topBrand,{<class 'bool'>},555,0.475578
brandCode,{<class 'str'>},933,0.799486


Initial observations:
* brandCode is not populated for every brand ID.  This means it's likely a brand that appears in a receipt might not have a record in the brands data schema


#### From my observations above, the first data quality concern I want to investigate further is the lack of brandCodes and barcodes for a large proportion of receipt item list lines:

In [12]:
#First:  Brandcodes

#Initialize spend and count totals
total_spend = 0
total_spend_no_barcode = 0
total_spend_no_brandcode = 0

total_count = 0
total_count_no_barcode = 0
total_count_no_brandcode = 0

for receipt_line in rewardsReceiptItemLines_flat: #Loop through receipt item lines
    #Set current price.  I'm prioritizing finalPrice over itemPrice over userFlaggedPrice based on 
    #the names, but this is a question for the stakeholder
    current_price = receipt_line.get("finalPrice")
    if current_price is None: current_price = receipt_line.get("itemPrice")
    if current_price is None: current_price = receipt_line.get("userFlaggedPrice")
    
    #Set current barcode
    curr_barcode = receipt_line.get("barcode")
    if curr_barcode is None: curr_barcode = receipt_line.get("userFlaggedBarcode")
    
    #Set current brandcode
    curr_brandcode = receipt_line.get("brandCode")
    
    #Lines missing a valid barcode 
    if curr_barcode == '4011' or curr_barcode is None and current_price is not None:
        total_spend_no_barcode += float(current_price) #current price is a string, so we must cast it
        total_count_no_barcode += 1
    
    #Lines missing a brandcode
    if curr_brandcode is None and current_price is not None:
        total_spend_no_brandcode += float(current_price)
        total_count_no_brandcode += 1
    if current_price is not None:
        total_spend += float(current_price)
        total_count +=1
        
BrandBarDF = pd.DataFrame([int(total_count),
                           str(round(total_spend,2)), 
                           total_spend_no_brandcode/total_spend,
                           total_count_no_brandcode/total_count,
                           total_spend_no_barcode/total_spend,
                           total_count_no_barcode/total_count
                           ])
BrandBarDF.index = ["Total Count",
                    "Total Spend",
                    "Proportion of Spend without a Brand Code",
                    "Proportion of Count without a Brand Code",
                    "Proportion of Spend without a valid barcode",
                    "Proportion of Count without a valid barcode"]
BrandBarDF.columns = ["Value"]

In [13]:
BrandBarDF

,Value
Total Count,6917
Total Spend,57362.53
Proportion of Spend without a Brand Code,0.675815
Proportion of Count without a Brand Code,0.624115
Proportion of Spend without a valid barcode,0.523621
Proportion of Count without a valid barcode,0.562672


### Summary of Data Quality Issue:

67.24% of receipt lines, accounting for 67.58% of spending, do not have a brand code and therefore cannot be associated with a brand.  This jeopardizes our ability to reliably answer questions 1,2,5, and 6 asked by the stakeholder because there are very likely brands that we cannot summarize data for, and therefore cannot say with confidence which brands have the most receipts scanned, highest spend, or most transactions.

The next data quality issue I would investigate is the possible discrepancy between sums of user flagged prices and total spending on receipts.  I won't be diving into this as part of this exercise for the sake of time, but it is still relevant to report to the stakeholder as a potential issue that I'll be following up on.

### Summary of questions for the stakeholder:

* What should be done with receipt lines with a barcode of 4011 and/or a missing brand code?
* Is it possible, at some point in the future, data will be added to the Users dataset with a role other than "consumer"?
* Is the signUpSource of individual users relevant for your business needs?
* Is item price, final price, or user flagged price the source of truth for the amount spent on an individual item?
* How can we reliably identify and remove test brands from the brands dataset?